In [783]:
import pandas as pd
import numpy as np

In [784]:
# DOE's list of days school was in session (includes remote school)

in_session_hs = pd.read_excel(r'/Users/ravram/Downloads/calendar.days.2015-2022.d110822.high.school.xlsx')
in_session_el = pd.read_excel(r'/Users/ravram/Downloads/calendar.days.2015-2022.d110822.elementary.school.xlsx')

In [785]:
# cleaning up the dataframes

in_session_hs = in_session_hs.rename(columns={in_session_hs.columns[0]: 'date'})
in_session_hs.drop(in_session_hs.tail(1).index, inplace=True)

in_session_el = in_session_el.rename(columns={in_session_el.columns[0]: 'date'})
in_session_el.drop(in_session_el.tail(1).index, inplace=True)

new_row = pd.DataFrame({'date':'2015-09-09'}, index =[0])
in_session_hs = pd.concat([new_row, in_session_hs]).reset_index(drop = True)
in_session_el = pd.concat([new_row, in_session_el]).reset_index(drop = True)

# converting all entries to datetime

in_session_hs['date'] = pd.to_datetime(in_session_hs['date'])
in_session_el['date'] = pd.to_datetime(in_session_el['date'])

In [786]:
# days of school for high school but not elementary school

hs_only = []

for day in in_session_hs['date'].to_list():
    if day not in in_session_el['date'].to_list():
        hs_only.append(pd.to_datetime(day))

In [787]:
# days of school for elementary school but not high school

elem_only = []

for day in in_session_el['date'].to_list():
    if day not in in_session_hs['date'].to_list():
        elem_only.append(pd.to_datetime(day))

In [788]:
# creating df of all days schools were in session across Elem and HS (starting in 2017-2018 SY)

in_session_total = pd.concat([in_session_hs, pd.DataFrame({'date': elem_only})], ignore_index=True)
in_session_total = in_session_total.sort_values(by = 'date')
in_session_total = in_session_total[in_session_total['date'] >= pd.to_datetime('2017-09-07')]

In [789]:
# adding a column indicating which schools were in session on a given date

in_session_total['school type'] = ''

for ind in in_session_total.index:
    if in_session_total['date'][ind] in hs_only:
        in_session_total['school type'][ind] = 'HS Only'
    elif in_session_total['date'][ind] in elem_only:
        in_session_total['school type'][ind] = 'Elem Only'
    else:
        in_session_total['school type'][ind] = 'HS/Elem'

/var/folders/0x/wbh6lcrn3t7046vw2zxtbrhxrc0v0g/T/ipykernel_6389/1468934571.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  in_session_total['school type'][ind] = 'HS/Elem'
/var/folders/0x/wbh6lcrn3t7046vw2zxtbrhxrc0v0g/T/ipykernel_6389/1468934571.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  in_session_total['school type'][ind] = 'Elem Only'
/var/folders/0x/wbh6lcrn3t7046vw2zxtbrhxrc0v0g/T/ipykernel_6389/1468934571.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

In [790]:
# creating df that contains only days both HS and elementary school students were physically in school, according to DOE

students_in_school = in_session_total[in_session_total['school type'] == 'HS/Elem']
students_in_school = students_in_school.drop(columns = ['school type'])

# removing days students were remote due to COVID

students_in_school = students_in_school[(students_in_school['date'] < pd.to_datetime('3/16/2020')) | (students_in_school['date'] > pd.to_datetime('9/13/2021'))]

In [791]:
students_in_school.to_csv('DOE School Bus Delay Dates.csv', index = False)